<a href="https://colab.research.google.com/github/human-ai2025/Elo-Merchant-Recommendation/blob/master/Elo-Merchant-Recommendation/Feature%20Engineering/featureEngineeringMMElo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importing the libraries

In [1]:
#Import Libraries 
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import gc
from contextlib import contextmanager
from pandas.core.common import SettingWithCopyWarning
import datetime
import time
import warnings
warnings.filterwarnings('ignore')
from scipy.stats import mode
import pickle
import datetime 

In [2]:
! [ -e /content ] && pip install -Uqq fastai  # upgrade fastai on colab

     |████████████████████████████████| 194kB 5.6MB/s 
     |████████████████████████████████| 61kB 5.2MB/s 


In [3]:
from fastai.torch_basics import *
from fastai.data.all import *

## Mounting Drive

In [2]:
#Mounting drive 
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Set up the path

In [3]:
%cd /content/drive/MyDrive/data

/content/drive/MyDrive/data


### View Current Items in the folder

In [6]:
!ls

 augmented_test.csv
 augmented_test_try1.csv
 augmented_test_try1_model_imputation.csv
 augmented_train.csv
 augmented_train_try1.csv
 augmented_train_try1_model_imputation.csv
 clf_cat2.sav
 clf_cat3.sav
 clf_name.sav
'Data Dictionary.xlsx'
 Data_Dictionary.xlsx
 dateTimeFeatures.csv
 file_name.csv
 hist_authorized_flag_enc.npy
 hist_category_1_enc.npy
 hist_category_3_enc.npy
 historical_transactions.csv
 historical_transactions_df.pkl
 hist_trans_preprocessed.csv
 knn_sal_12.sav
 knn_sal_3.sav
 knn_sal_6.sav
 LR_cat_2.sav
 mer_authorized_flag_enc.npy
 mer_category_1_enc.npy
 mer_category_3_enc.npy
 merchant_category_1_enc.npy
 merchant_category_4_enc.npy
 merchant_most_recent_purchases_range_enc.npy
 merchant_most_recent_sales_range_enc.npy
 merchants.csv
 merchants_preprocessed.csv
 merged_transactions_with_merchants.csv
 new_merchant_authorized_flag_enc.npy
 new_merchant_category_1_enc.npy
 new_merchant_df.pkl
 newMerchants_trans_preprocessed.csv
 new_merchant_transactions.csv
 sa

## Feature Engineering 

1. Feature Engineering involves ideating, selecting, and creating useful features in the datasets, to help fully identify the potential fruitful connections, correlations, and patterns to explore in your machine learning models. 
2. Interpreating dates and adding new feature dependant on date and time.
3. Statiscal aggregates on numerical and categorical columns. 
4. Feature interactions between new merchant transaction and historical transactions.


## Helper Functions

In [4]:
def reduce_mem_usage(df, verbose=True):
  """ Helps to reduce memory usage """
  numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
  start_mem = df.memory_usage().sum() / 1024**2
  for col in df.columns:
      col_type = df[col].dtypes
      if col_type in numerics:
          c_min = df[col].min()
          c_max = df[col].max()
          if str(col_type)[:3] == 'int':
              if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                  df[col] = df[col].astype(np.int8)
              elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                  df[col] = df[col].astype(np.int16)
              elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                  df[col] = df[col].astype(np.int32)
              elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                  df[col] = df[col].astype(np.int64)
          else:
              if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                  df[col] = df[col].astype(np.float16)
              elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                  df[col] = df[col].astype(np.float32)
              else:
                  df[col] = df[col].astype(np.float64)

  end_mem = df.memory_usage().sum() / 1024**2
  print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
  print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

  return df

In [8]:
def add_datepart(df, fldname, drop=True, time=False):
    """
    Helper function that adds columns relevant to a date.
    -----------------------------------------------------
    Args:
          df: The data frame on which you want to perform operations 
          fldname: The name of the column
          drop: Drop the parent column from the data frame 
          time: Add attributes such as hour, minute and second
    ------------------------------------------------------
    This code is taken from the below link 
    Refer:-https://gist.github.com/adidoit/d32f6117647b0e8564edc6507f9e3625
    Refer:-https://colab.research.google.com/github/fastai/fastai/blob/master/nbs/40_tabular.core.ipynb#scrollTo=qt7-T33VVNwx
    """
    fld = df[fldname]
    fld_dtype = fld.dtype
    if isinstance(fld_dtype, pd.core.dtypes.dtypes.DatetimeTZDtype):
        fld_dtype = np.datetime64

    if not np.issubdtype(fld_dtype, np.datetime64):
        df[fldname] = fld = pd.to_datetime(fld, infer_datetime_format=True)
    targ_pre = re.sub('[Dd]ate$', '', fldname)
    attr = ['Year', 'Month', 'Week', 'Day', 'Dayofweek', 'Dayofyear',
            'Is_month_end', 'Is_month_start', 'Is_quarter_end', 'Is_quarter_start', 'Is_year_end', 'Is_year_start']
    if time: attr = attr + ['Hour', 'Minute', 'Second']
    #https://www.programiz.com/python-programming/methods/built-in/getattr
    for n in attr: df[targ_pre + n] = getattr(fld.dt, n.lower())
    df[targ_pre + 'Elapsed'] = fld.astype(np.int64) // 10 ** 9
    if drop: df.drop(fldname, axis=1, inplace=True)

In [9]:
def dateTimeFeatures(df):
  """

    To retuurn a dataframe with new date time features 
    ---------------------------------------------------------
    Args:
      df:- The dataframe on which we want to do 
    ---------------------------------------------------------
    Returns:
    A dataframe with date time features 
    ---------------------------------------------------------
    To DO:
      1. Add holidays 

  """

  # basic date time features
  df['purchasedWeekend'] = (df.purchase_Dayofweek >=5).astype(int)
  df['purchasedWeekday'] = (df.purchase_Dayofweek <5).astype(int)
  #difference in months from today
  df['monthDiff'] = ((datetime.today() - df['purchase_date']).dt.days) // 30
  #month lag to reference date
  df['monthDiff'] += df['month_lag']

  return df

In [11]:
def nullValues(df):
  """ Find the columns with null values """
  columns_with_na = [var for var in df.columns if df[var].isnull().mean()  > 0]
  #lets find out the percentage of observations missing per variable

  #calculate the percentage of missing 
  data_na = df[columns_with_na].isnull().mean()

  #transform the array to dataframe 
  data_na = pd.DataFrame(data_na.reset_index())

  #add names to the dataframe 
  data_na.columns = ['col','percentage_na']

  #oreder the dataframe acc to percentage 
  data_na.sort_values(by = 'percentage_na',ascending = False, inplace = True)

  #show
  data_na
  return data_na

## Merge and appply

In [13]:
#read the files 

histDf = reduce_mem_usage(pd.read_csv("hist_trans_preprocessed.csv",parse_dates=['purchase_date']))

newdf = reduce_mem_usage(pd.read_csv("newMerchants_trans_preprocessed.csv",parse_dates=['purchase_date']) )

Memory usage after optimization is: 1166.08 MB
Decreased by 62.5%
Memory usage after optimization is: 74.88 MB
Decreased by 64.3%


In [14]:
#merge the two files 
transactions = histDf.append(newdf, ignore_index=True)

In [15]:

del histDf, newdf
gc.collect()

0

In [16]:
add_datepart(transactions, 'purchase_date', drop=False, time=True)

In [17]:
transactions.head()

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id,purchase_Year,purchase_Month,purchase_Week,purchase_Day,purchase_Dayofweek,purchase_Dayofyear,purchase_Is_month_end,purchase_Is_month_start,purchase_Is_quarter_end,purchase_Is_quarter_start,purchase_Is_year_end,purchase_Is_year_start,purchase_Hour,purchase_Minute,purchase_Second,purchase_Elapsed
0,1,C_ID_4e6213e9bc,88,0,0,0,80,M_ID_e020e9b302,-8,-0.703331,2017-06-25 15:33:07,1.0,16,37,2017,6,25,25,6,176,False,False,False,False,False,False,15,33,7,1498404787
1,1,C_ID_4e6213e9bc,88,0,0,0,367,M_ID_86ec983688,-7,-0.733128,2017-07-15 12:10:45,1.0,16,16,2017,7,28,15,5,196,False,False,False,False,False,False,12,10,45,1500120645
2,1,C_ID_4e6213e9bc,88,0,0,0,80,M_ID_979ed661fc,-6,-0.720386,2017-08-09 22:04:29,1.0,16,37,2017,8,32,9,2,221,False,False,False,False,False,False,22,4,29,1502316269
3,1,C_ID_4e6213e9bc,88,0,0,0,560,M_ID_e6d5ae8ea6,-5,-0.735352,2017-09-02 10:06:26,1.0,16,34,2017,9,35,2,5,245,False,False,False,False,False,False,10,6,26,1504346786
4,1,C_ID_4e6213e9bc,88,0,0,0,80,M_ID_e020e9b302,-11,-0.722865,2017-03-10 01:14:19,1.0,16,37,2017,3,10,10,4,69,False,False,False,False,False,False,1,14,19,1489108459


In [18]:
mapping = {False:0, True:1}
transactions.replace({"purchase_Is_month_end": mapping,
                      "purchase_Is_month_start": mapping,
                      "purchase_Is_quarter_end": mapping,
                      "purchase_Is_quarter_start": mapping,
                      "purchase_Is_year_end": mapping,
                      "purchase_Is_year_start": mapping},
                     inplace = True)

In [19]:
transactions = dateTimeFeatures(transactions)

In [20]:
transactions.head().T

,0,1,2,3,4
authorized_flag,1,1,1,1,1
card_id,C_ID_4e6213e9bc,C_ID_4e6213e9bc,C_ID_4e6213e9bc,C_ID_4e6213e9bc,C_ID_4e6213e9bc
city_id,88,88,88,88,88
category_1,0,0,0,0,0
installments,0,0,0,0,0
category_3,0,0,0,0,0
merchant_category_id,80,367,80,560,80
merchant_id,M_ID_e020e9b302,M_ID_86ec983688,M_ID_979ed661fc,M_ID_e6d5ae8ea6,M_ID_e020e9b302
month_lag,-8,-7,-6,-5,-11
purchase_amount,-0.703331,-0.733128,-0.720386,-0.735352,-0.722865


### Purchase Amount 
The purchase amount column is normalized and heavily trannsforme. 


In [21]:
#Refer:-https://www.kaggle.com/raddar/towards-de-anonymizing-the-data-some-insights

transactions['purchase_amount'] = np.round(transactions['purchase_amount'] / 0.00150265118 + 497.06,2)

In [22]:
transactions.purchase_amount

0           29.00
1            9.17
2           17.65
3            7.69
4           16.00
            ...  
31075387    30.11
31075388    34.99
31075389    83.73
31075390    60.01
31075391     5.09
Name: purchase_amount, Length: 31075392, dtype: float64

In [23]:
#find the percentile value to get the outliners 
for i in range(90, 101):
  print("{} percentile value is {:3.3f}".format(i, np.percentile(transactions['purchase_amount'], i)))

90 percentile value is 215.000
91 percentile value is 236.310
92 percentile value is 260.000
93 percentile value is 294.900
94 percentile value is 331.830
95 percentile value is 389.200
96 percentile value is 467.650
97 percentile value is 590.000
98 percentile value is 800.070
99 percentile value is 1322.000
100 percentile value is 4000000017.910


In [24]:
# Remove the 100th percentile 
transactions = transactions.drop(transactions[transactions['purchase_amount'] == 4000000017.910].index)

In [25]:
# one hot encoding 
transactions = pd.get_dummies(transactions, columns=['purchasedWeekday', 'purchase_Month', 'purchasedWeekend', 'purchase_Dayofweek'])
transactions.head()

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id,purchase_Year,purchase_Week,purchase_Day,purchase_Dayofyear,purchase_Is_month_end,purchase_Is_month_start,purchase_Is_quarter_end,purchase_Is_quarter_start,purchase_Is_year_end,purchase_Is_year_start,purchase_Hour,purchase_Minute,purchase_Second,purchase_Elapsed,monthDiff,purchasedWeekday_0,purchasedWeekday_1,purchase_Month_1,purchase_Month_2,purchase_Month_3,purchase_Month_4,purchase_Month_5,purchase_Month_6,purchase_Month_7,purchase_Month_8,purchase_Month_9,purchase_Month_10,purchase_Month_11,purchase_Month_12,purchasedWeekend_0,purchasedWeekend_1,purchase_Dayofweek_0,purchase_Dayofweek_1,purchase_Dayofweek_2,purchase_Dayofweek_3,purchase_Dayofweek_4,purchase_Dayofweek_5,purchase_Dayofweek_6
0,1,C_ID_4e6213e9bc,88,0,0,0,80,M_ID_e020e9b302,-8,29.00,2017-06-25 15:33:07,1.0,16,37,2017,25,25,176,0,0,0,0,0,0,15,33,7,1498404787,36,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1
1,1,C_ID_4e6213e9bc,88,0,0,0,367,M_ID_86ec983688,-7,9.17,2017-07-15 12:10:45,1.0,16,16,2017,28,15,196,0,0,0,0,0,0,12,10,45,1500120645,36,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0
2,1,C_ID_4e6213e9bc,88,0,0,0,80,M_ID_979ed661fc,-6,17.65,2017-08-09 22:04:29,1.0,16,37,2017,32,9,221,0,0,0,0,0,0,22,4,29,1502316269,36,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0
3,1,C_ID_4e6213e9bc,88,0,0,0,560,M_ID_e6d5ae8ea6,-5,7.69,2017-09-02 10:06:26,1.0,16,34,2017,35,2,245,0,0,0,0,0,0,10,6,26,1504346786,37,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0
4,1,C_ID_4e6213e9bc,88,0,0,0,80,M_ID_e020e9b302,-11,16.00,2017-03-10 01:14:19,1.0,16,37,2017,10,10,69,0,0,0,0,0,0,1,14,19,1489108459,37,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0


#### Find the aggregations on date time features

In [26]:
transactions.purchase_Year.value_counts().idxmax()

2017

In [27]:
transactions['purchase_date_temp'] = transactions.purchase_date.dt.date

In [28]:
def mode(series):
    """Most common element in a series"""
    tmode = series.value_counts().idxmax()
    return tmode
        
def period(series):
    """Period of a series (max-min)"""
    return series.max() - series.min()

def mean_diff(series):
    """Mean difference between consecutive items in a series"""
    ss = series.sort_values()
    return (ss - ss.shift()).mean()


common_aggs = ['sum', 'mean', 'std']

In [29]:
aggs = {
      'purchase_Year' : mode,
      'purchase_Week' : common_aggs,
      'purchase_Day' : common_aggs,
      'purchase_Dayofyear' : common_aggs, 
      'purchase_Is_month_end' : common_aggs,
      'purchase_Is_month_start' : common_aggs, 
      'purchase_Is_quarter_end' : common_aggs,
      'purchase_Is_quarter_start' : common_aggs, 
      'purchase_Is_year_end' : common_aggs,
      'purchase_Is_year_start' :common_aggs, 
      'purchase_Hour' : common_aggs, 
      'purchase_Minute' : common_aggs,
      'purchase_Second' : common_aggs, 
      'purchase_Elapsed' : common_aggs, 
      'purchasedWeekday_0' : common_aggs, 
      'purchasedWeekday_1': common_aggs, 
      'purchase_Month_1': common_aggs,
      'purchase_Month_2': common_aggs, 
      'purchase_Month_3': common_aggs, 
      'purchase_Month_4': common_aggs,
      'purchase_Month_5': common_aggs, 
      'purchase_Month_6': common_aggs, 
      'purchase_Month_7': common_aggs,
      'purchase_Month_8': common_aggs, 
      'purchase_Month_9': common_aggs, 
      'purchase_Month_10': common_aggs,
      'purchase_Month_11': common_aggs, 
      'purchase_Month_12': common_aggs, 
      'purchasedWeekend_0': common_aggs,
      'purchasedWeekend_1': common_aggs, 
      'purchase_Dayofweek_0': common_aggs, 
      'purchase_Dayofweek_1': common_aggs,
      'purchase_Dayofweek_2': common_aggs, 
      'purchase_Dayofweek_3': common_aggs, 
      'purchase_Dayofweek_4': common_aggs,
      'purchase_Dayofweek_5': common_aggs, 
      'purchase_Dayofweek_6': common_aggs,
      'purchase_date_temp': [period , mean_diff, 'min', 'max',],
      'monthDiff' :[period , mean_diff]
}

In [30]:
%%time
dateTimeFeatures = transactions.groupby('card_id').agg(aggs)
#rename 
dateTimeFeatures.columns = ['_'.join(col).strip() 
                           for col in dateTimeFeatures.columns.values]

CPU times: user 18min 24s, sys: 3.14 s, total: 18min 27s
Wall time: 18min 27s


In [31]:
dateTimeFeatures

,purchase_Year_mode,purchase_Week_sum,purchase_Week_mean,purchase_Week_std,purchase_Day_sum,purchase_Day_mean,purchase_Day_std,purchase_Dayofyear_sum,purchase_Dayofyear_mean,purchase_Dayofyear_std,purchase_Is_month_end_sum,purchase_Is_month_end_mean,purchase_Is_month_end_std,purchase_Is_month_start_sum,purchase_Is_month_start_mean,purchase_Is_month_start_std,purchase_Is_quarter_end_sum,purchase_Is_quarter_end_mean,purchase_Is_quarter_end_std,purchase_Is_quarter_start_sum,purchase_Is_quarter_start_mean,purchase_Is_quarter_start_std,purchase_Is_year_end_sum,purchase_Is_year_end_mean,purchase_Is_year_end_std,purchase_Is_year_start_sum,purchase_Is_year_start_mean,purchase_Is_year_start_std,purchase_Hour_sum,purchase_Hour_mean,purchase_Hour_std,purchase_Minute_sum,purchase_Minute_mean,purchase_Minute_std,purchase_Second_sum,purchase_Second_mean,purchase_Second_std,purchase_Elapsed_sum,purchase_Elapsed_mean,purchase_Elapsed_std,...,purchase_Month_10_std,purchase_Month_11_sum,purchase_Month_11_mean,purchase_Month_11_std,purchase_Month_12_sum,purchase_Month_12_mean,purchase_Month_12_std,purchasedWeekend_0_sum,purchasedWeekend_0_mean,purchasedWeekend_0_std,purchasedWeekend_1_sum,purchasedWeekend_1_mean,purchasedWeekend_1_std,purchase_Dayofweek_0_sum,purchase_Dayofweek_0_mean,purchase_Dayofweek_0_std,purchase_Dayofweek_1_sum,purchase_Dayofweek_1_mean,purchase_Dayofweek_1_std,purchase_Dayofweek_2_sum,purchase_Dayofweek_2_mean,purchase_Dayofweek_2_std,purchase_Dayofweek_3_sum,purchase_Dayofweek_3_mean,purchase_Dayofweek_3_std,purchase_Dayofweek_4_sum,purchase_Dayofweek_4_mean,purchase_Dayofweek_4_std,purchase_Dayofweek_5_sum,purchase_Dayofweek_5_mean,purchase_Dayofweek_5_std,purchase_Dayofweek_6_sum,purchase_Dayofweek_6_mean,purchase_Dayofweek_6_std,purchase_date_temp_period,purchase_date_temp_mean_diff,purchase_date_temp_min,purchase_date_temp_max,monthDiff_period,monthDiff_mean_diff
card_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
C_ID_00007093c1,2017,3836,25.403974,14.296936,1986,13.152318,9.066237,26408,174.887417,100.681646,3,0.019868,0.140009,10,0.066225,0.249503,0,0.000000,0.000000,2,0.013245,0.114703,0,0.000000,0.000000,1,0.006623,0.081379,2175,14.403974,4.209796,4552,30.145695,17.559953,4384,29.033113,17.460973,227001125104,1.503319e+09,9.232254e+06,...,0.317134,6.0,0.039735,0.195986,12.0,0.079470,0.271371,126.0,0.834437,0.372925,25.0,0.165563,0.372925,43.0,0.284768,0.452806,17.0,0.112583,0.317134,25.0,0.165563,0.372925,25.0,0.165563,0.372925,16.0,0.105960,0.308811,21.0,0.139073,0.347174,4.0,0.026490,0.161122,419 days,2 days 19:02:24,2017-02-14,2018-04-09,1,0.006667
C_ID_0001238066,2018,4020,26.979866,20.460730,2459,16.503356,8.621645,27855,186.946309,143.940152,1,0.006711,0.081923,3,0.020134,0.140933,0,0.000000,0.000000,1,0.006711,0.081923,0,0.000000,0.000000,0,0.000000,0.000000,2208,14.818792,5.862217,4269,28.651007,16.055615,4270,28.657718,19.364139,225799834810,1.515435e+09,4.435143e+06,...,0.262369,23.0,0.154362,0.362514,37.0,0.248322,0.433497,85.0,0.570470,0.496679,64.0,0.429530,0.496679,11.0,0.073826,0.262369,20.0,0.134228,0.342047,12.0,0.080537,0.273040,19.0,0.127517,0.334676,23.0,0.154362,0.362514,41.0,0.275168,0.448105,23.0,0.154362,0.362514,214 days,1 days 10:42:09.729729729,2017-09-28,2018-04-30,1,0.006757
C_ID_0001506ef0,2017,1811,26.632353,19.511883,830,12.205882,7.616868,12565,184.779412,136.566079,1,0.014706,0.121268,1,0.014706,0.121268,1,0.014706,0.121268,0,0.000000,0.000000,0,0.000000,0.000000,0,0.000000,0.000000,863,12.691176,3.342433,1993,29.308824,17.999384,2066,30.382353,17.540596,102415567646,1.506111e+09,1.127141e+07,...,0.206883,7.0,0.102941,0.306141,18.0,0.264706,0.444457,36.0,0.529412,0.502845,32.0,0.470588,0.502845,4.0,0.058824,0.237044,9.0,0.132353,0.341394,6.0,0.088235,0.285746,10.0,0.147059,0.356798,7.0,0.102941,0.306141,28.0,0.411765,0.495812,4.0,0.058824,0.237044,432 days,6 days 10:44:46.567164179,2017-01-14,2018-03-22,1,0.014925
C_ID_0001793786,2017,7361,29

In [32]:
from datetime import date 

In [33]:
# well need to convert purchase_date_temp_period, purchase_date_temp_mean_diff, purchase_date_temp_min, purchase_date_temp_max
dateTimeFeatures.purchase_date_temp_period = dateTimeFeatures.purchase_date_temp_period.dt.days
dateTimeFeatures.purchase_date_temp_mean_diff = dateTimeFeatures.purchase_date_temp_mean_diff.dt.total_seconds()/3600
dateTimeFeatures.purchase_date_temp_min = (date.today() - dateTimeFeatures['purchase_date_temp_min']).dt.days
dateTimeFeatures.purchase_date_temp_max = (date.today() - dateTimeFeatures['purchase_date_temp_max']).dt.days

In [34]:
dateTimeFeatures

,purchase_Year_mode,purchase_Week_sum,purchase_Week_mean,purchase_Week_std,purchase_Day_sum,purchase_Day_mean,purchase_Day_std,purchase_Dayofyear_sum,purchase_Dayofyear_mean,purchase_Dayofyear_std,purchase_Is_month_end_sum,purchase_Is_month_end_mean,purchase_Is_month_end_std,purchase_Is_month_start_sum,purchase_Is_month_start_mean,purchase_Is_month_start_std,purchase_Is_quarter_end_sum,purchase_Is_quarter_end_mean,purchase_Is_quarter_end_std,purchase_Is_quarter_start_sum,purchase_Is_quarter_start_mean,purchase_Is_quarter_start_std,purchase_Is_year_end_sum,purchase_Is_year_end_mean,purchase_Is_year_end_std,purchase_Is_year_start_sum,purchase_Is_year_start_mean,purchase_Is_year_start_std,purchase_Hour_sum,purchase_Hour_mean,purchase_Hour_std,purchase_Minute_sum,purchase_Minute_mean,purchase_Minute_std,purchase_Second_sum,purchase_Second_mean,purchase_Second_std,purchase_Elapsed_sum,purchase_Elapsed_mean,purchase_Elapsed_std,...,purchase_Month_10_std,purchase_Month_11_sum,purchase_Month_11_mean,purchase_Month_11_std,purchase_Month_12_sum,purchase_Month_12_mean,purchase_Month_12_std,purchasedWeekend_0_sum,purchasedWeekend_0_mean,purchasedWeekend_0_std,purchasedWeekend_1_sum,purchasedWeekend_1_mean,purchasedWeekend_1_std,purchase_Dayofweek_0_sum,purchase_Dayofweek_0_mean,purchase_Dayofweek_0_std,purchase_Dayofweek_1_sum,purchase_Dayofweek_1_mean,purchase_Dayofweek_1_std,purchase_Dayofweek_2_sum,purchase_Dayofweek_2_mean,purchase_Dayofweek_2_std,purchase_Dayofweek_3_sum,purchase_Dayofweek_3_mean,purchase_Dayofweek_3_std,purchase_Dayofweek_4_sum,purchase_Dayofweek_4_mean,purchase_Dayofweek_4_std,purchase_Dayofweek_5_sum,purchase_Dayofweek_5_mean,purchase_Dayofweek_5_std,purchase_Dayofweek_6_sum,purchase_Dayofweek_6_mean,purchase_Dayofweek_6_std,purchase_date_temp_period,purchase_date_temp_mean_diff,purchase_date_temp_min,purchase_date_temp_max,monthDiff_period,monthDiff_mean_diff
card_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
C_ID_00007093c1,2017,3836,25.403974,14.296936,1986,13.152318,9.066237,26408,174.887417,100.681646,3,0.019868,0.140009,10,0.066225,0.249503,0,0.000000,0.000000,2,0.013245,0.114703,0,0.000000,0.000000,1,0.006623,0.081379,2175,14.403974,4.209796,4552,30.145695,17.559953,4384,29.033113,17.460973,227001125104,1.503319e+09,9.232254e+06,...,0.317134,6.0,0.039735,0.195986,12.0,0.079470,0.271371,126.0,0.834437,0.372925,25.0,0.165563,0.372925,43.0,0.284768,0.452806,17.0,0.112583,0.317134,25.0,0.165563,0.372925,25.0,0.165563,0.372925,16.0,0.105960,0.308811,21.0,0.139073,0.347174,4.0,0.026490,0.161122,419,67.040000,1464,1045,1,0.006667
C_ID_0001238066,2018,4020,26.979866,20.460730,2459,16.503356,8.621645,27855,186.946309,143.940152,1,0.006711,0.081923,3,0.020134,0.140933,0,0.000000,0.000000,1,0.006711,0.081923,0,0.000000,0.000000,0,0.000000,0.000000,2208,14.818792,5.862217,4269,28.651007,16.055615,4270,28.657718,19.364139,225799834810,1.515435e+09,4.435143e+06,...,0.262369,23.0,0.154362,0.362514,37.0,0.248322,0.433497,85.0,0.570470,0.496679,64.0,0.429530,0.496679,11.0,0.073826,0.262369,20.0,0.134228,0.342047,12.0,0.080537,0.273040,19.0,0.127517,0.334676,23.0,0.154362,0.362514,41.0,0.275168,0.448105,23.0,0.154362,0.362514,214,34.702703,1238,1024,1,0.006757
C_ID_0001506ef0,2017,1811,26.632353,19.511883,830,12.205882,7.616868,12565,184.779412,136.566079,1,0.014706,0.121268,1,0.014706,0.121268,1,0.014706,0.121268,0,0.000000,0.000000,0,0.000000,0.000000,0,0.000000,0.000000,863,12.691176,3.342433,1993,29.308824,17.999384,2066,30.382353,17.540596,102415567646,1.506111e+09,1.127141e+07,...,0.206883,7.0,0.102941,0.306141,18.0,0.264706,0.444457,36.0,0.529412,0.502845,32.0,0.470588,0.502845,4.0,0.058824,0.237044,9.0,0.132353,0.341394,6.0,0.088235,0.285746,10.0,0.147059,0.356798,7.0,0.102941,0.306141,28.0,0.411765,0.495812,4.0,0.058824,0.237044,432,154.746269,1495,1063,1,0.014925
C_ID_0001793786,2017,7361,29.801619,11.855375,4213,17.056680,8.453894,50985,206.417004,83.052292,7,0.028340,0.166279

In [35]:
def remove_noninformative(df):
    """Remove non-informative columns (all nan, or all same value)"""
    for col in df:
        if df[col].isnull().all():
            print('Removing '+col+' (all NaN)')
            del df[col]
        elif df[col].nunique()<2:
            print('Removing '+col+' (only 1 unique value)')
            del df[col]

remove_noninformative(dateTimeFeatures)
gc.collect()

0

In [36]:
dateTimeFeatures.to_csv('dateTimeFeatures.csv')

In [37]:
dateTimeFeatures

,purchase_Year_mode,purchase_Week_sum,purchase_Week_mean,purchase_Week_std,purchase_Day_sum,purchase_Day_mean,purchase_Day_std,purchase_Dayofyear_sum,purchase_Dayofyear_mean,purchase_Dayofyear_std,purchase_Is_month_end_sum,purchase_Is_month_end_mean,purchase_Is_month_end_std,purchase_Is_month_start_sum,purchase_Is_month_start_mean,purchase_Is_month_start_std,purchase_Is_quarter_end_sum,purchase_Is_quarter_end_mean,purchase_Is_quarter_end_std,purchase_Is_quarter_start_sum,purchase_Is_quarter_start_mean,purchase_Is_quarter_start_std,purchase_Is_year_end_sum,purchase_Is_year_end_mean,purchase_Is_year_end_std,purchase_Is_year_start_sum,purchase_Is_year_start_mean,purchase_Is_year_start_std,purchase_Hour_sum,purchase_Hour_mean,purchase_Hour_std,purchase_Minute_sum,purchase_Minute_mean,purchase_Minute_std,purchase_Second_sum,purchase_Second_mean,purchase_Second_std,purchase_Elapsed_sum,purchase_Elapsed_mean,purchase_Elapsed_std,...,purchase_Month_10_std,purchase_Month_11_sum,purchase_Month_11_mean,purchase_Month_11_std,purchase_Month_12_sum,purchase_Month_12_mean,purchase_Month_12_std,purchasedWeekend_0_sum,purchasedWeekend_0_mean,purchasedWeekend_0_std,purchasedWeekend_1_sum,purchasedWeekend_1_mean,purchasedWeekend_1_std,purchase_Dayofweek_0_sum,purchase_Dayofweek_0_mean,purchase_Dayofweek_0_std,purchase_Dayofweek_1_sum,purchase_Dayofweek_1_mean,purchase_Dayofweek_1_std,purchase_Dayofweek_2_sum,purchase_Dayofweek_2_mean,purchase_Dayofweek_2_std,purchase_Dayofweek_3_sum,purchase_Dayofweek_3_mean,purchase_Dayofweek_3_std,purchase_Dayofweek_4_sum,purchase_Dayofweek_4_mean,purchase_Dayofweek_4_std,purchase_Dayofweek_5_sum,purchase_Dayofweek_5_mean,purchase_Dayofweek_5_std,purchase_Dayofweek_6_sum,purchase_Dayofweek_6_mean,purchase_Dayofweek_6_std,purchase_date_temp_period,purchase_date_temp_mean_diff,purchase_date_temp_min,purchase_date_temp_max,monthDiff_period,monthDiff_mean_diff
card_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
C_ID_00007093c1,2017,3836,25.403974,14.296936,1986,13.152318,9.066237,26408,174.887417,100.681646,3,0.019868,0.140009,10,0.066225,0.249503,0,0.000000,0.000000,2,0.013245,0.114703,0,0.000000,0.000000,1,0.006623,0.081379,2175,14.403974,4.209796,4552,30.145695,17.559953,4384,29.033113,17.460973,227001125104,1.503319e+09,9.232254e+06,...,0.317134,6.0,0.039735,0.195986,12.0,0.079470,0.271371,126.0,0.834437,0.372925,25.0,0.165563,0.372925,43.0,0.284768,0.452806,17.0,0.112583,0.317134,25.0,0.165563,0.372925,25.0,0.165563,0.372925,16.0,0.105960,0.308811,21.0,0.139073,0.347174,4.0,0.026490,0.161122,419,67.040000,1464,1045,1,0.006667
C_ID_0001238066,2018,4020,26.979866,20.460730,2459,16.503356,8.621645,27855,186.946309,143.940152,1,0.006711,0.081923,3,0.020134,0.140933,0,0.000000,0.000000,1,0.006711,0.081923,0,0.000000,0.000000,0,0.000000,0.000000,2208,14.818792,5.862217,4269,28.651007,16.055615,4270,28.657718,19.364139,225799834810,1.515435e+09,4.435143e+06,...,0.262369,23.0,0.154362,0.362514,37.0,0.248322,0.433497,85.0,0.570470,0.496679,64.0,0.429530,0.496679,11.0,0.073826,0.262369,20.0,0.134228,0.342047,12.0,0.080537,0.273040,19.0,0.127517,0.334676,23.0,0.154362,0.362514,41.0,0.275168,0.448105,23.0,0.154362,0.362514,214,34.702703,1238,1024,1,0.006757
C_ID_0001506ef0,2017,1811,26.632353,19.511883,830,12.205882,7.616868,12565,184.779412,136.566079,1,0.014706,0.121268,1,0.014706,0.121268,1,0.014706,0.121268,0,0.000000,0.000000,0,0.000000,0.000000,0,0.000000,0.000000,863,12.691176,3.342433,1993,29.308824,17.999384,2066,30.382353,17.540596,102415567646,1.506111e+09,1.127141e+07,...,0.206883,7.0,0.102941,0.306141,18.0,0.264706,0.444457,36.0,0.529412,0.502845,32.0,0.470588,0.502845,4.0,0.058824,0.237044,9.0,0.132353,0.341394,6.0,0.088235,0.285746,10.0,0.147059,0.356798,7.0,0.102941,0.306141,28.0,0.411765,0.495812,4.0,0.058824,0.237044,432,154.746269,1495,1063,1,0.014925
C_ID_0001793786,2017,7361,29.801619,11.855375,4213,17.056680,8.453894,50985,206.417004,83.052292,7,0.028340,0.166279

## Feature engineering on other features 

In [38]:
common_aggs = ['sum', 'mean', 'std']
common_aggs1 = ['sum', 'mean', 'min', 'max', 'std']
aggs = {
      'authorized_flag' : common_aggs,
      'category_1' : common_aggs,
      'category_2' : common_aggs,
      'category_3' : common_aggs, 
      'month_lag' : common_aggs1, 
      'installments' : common_aggs1,
      'purchase_amount' : common_aggs1
}

In [39]:
%%time
otherFeatures = transactions.groupby('card_id').agg(aggs)
#rename 
otherFeatures.columns = ['_'.join(col).strip() 
                           for col in otherFeatures.columns.values]

CPU times: user 12.1 s, sys: 58.6 ms, total: 12.2 s
Wall time: 12.2 s


In [40]:
otherFeatures

,authorized_flag_sum,authorized_flag_mean,authorized_flag_std,category_1_sum,category_1_mean,category_1_std,category_2_sum,category_2_mean,category_2_std,category_3_sum,category_3_mean,category_3_std,month_lag_sum,month_lag_mean,month_lag_min,month_lag_max,month_lag_std,installments_sum,installments_mean,installments_min,installments_max,installments_std,purchase_amount_sum,purchase_amount_mean,purchase_amount_min,purchase_amount_max,purchase_amount_std
card_id,,,,,,,,,,,,,,,,,,,,,,,,,,,
C_ID_00007093c1,116.0,0.768212,0.423379,28.0,0.185430,0.389940,397.0,2.628906,0.813354,175.0,1.158940,0.366837,-868.0,-5.748344,-12,2,3.546301,194,1.284768,1,6,0.760504,23032.24,152.531391,12.00,1500.00,197.409986
C_ID_0001238066,146.0,0.979866,0.140933,4.0,0.026846,0.162177,241.0,1.617188,1.450057,181.0,1.214765,0.473112,-188.0,-1.261745,-5,2,1.690178,240,1.610738,-1,10,1.596920,15966.41,107.157114,4.12,1008.22,124.426646
C_ID_0001506ef0,64.0,0.941176,0.237044,0.0,0.000000,0.000000,200.0,2.941406,0.340428,1.0,0.014706,0.121268,-317.0,-4.661765,-13,1,4.290254,1,0.014706,0,1,0.121268,9809.72,144.260588,4.27,1491.00,310.398216
C_ID_0001793786,220.0,0.890688,0.312664,2.0,0.008097,0.089801,375.0,1.518555,0.679754,5.0,0.020243,0.141116,-678.0,-2.744939,-9,2,2.657197,5,0.020243,0,1,0.141116,98140.69,397.330729,1.00,3527.80,565.961144
C_ID_000183fdda,148.0,0.954839,0.208331,4.0,0.025806,0.159071,444.0,2.865234,0.547455,191.0,1.232258,0.494376,-339.0,-2.187097,-6,2,2.066389,280,1.806452,-1,10,2.070500,26847.17,173.207548,6.00,2337.00,338.702256
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
C_ID_ffff1d9928,13.0,0.812500,0.403113,3.0,0.187500,0.403113,52.0,3.250000,1.341641,17.0,1.062500,0.442531,-18.0,-1.125000,-4,2,1.995829,27,1.687500,-1,10,2.495830,1339.23,83.701875,0.01,500.00,143.970549
C_ID_ffff579d3a,100.0,0.869565,0.338255,0.0,0.000000,0.000000,119.0,1.035156,0.373002,3.0,0.026087,0.160092,-425.0,-3.695652,-6,1,2.018224,3,0.026087,0,1,0.160092,3285.90,28.573043,5.10,170.00,27.974194
C_ID_ffff756266,15.0,0.600000,0.500000,3.0,0.120000,0.331662,25.0,1.000000,0.000000,37.0,1.480000,0.585947,-204.0,-8.160000,-13,2,3.975760,79,3.160000,-1,12,3.578640,9014.03,360.561200,0.50,1299.91,346.523658


In [41]:
otherFeatures.to_csv('otherFeatures.csv')

## Merchants file 

In [42]:
merchants = pd.read_csv("merchants_preprocessed.csv") 
merchants.columns = 'mer_' + merchants.columns.values
merchants.rename(columns = {'mer_merchant_id':'merchant_id'}, inplace = True) 

In [45]:
requiredTrans = transactions[['card_id', 'merchant_id']]

In [46]:
merTrans = pd.merge(requiredTrans, merchants, on='merchant_id', how='left')

In [48]:
del requiredTrans, merchants
gc.collect()

737

In [47]:
merTrans.head().T

,0,1,2,3,4
card_id,C_ID_4e6213e9bc,C_ID_4e6213e9bc,C_ID_4e6213e9bc,C_ID_4e6213e9bc,C_ID_4e6213e9bc
merchant_id,M_ID_e020e9b302,M_ID_86ec983688,M_ID_979ed661fc,M_ID_e6d5ae8ea6,M_ID_e020e9b302
mer_merchant_group_id,35,2084,27369,24104,35
mer_merchant_category_id,80,367,80,560,80
mer_subsector_id,37,16,37,34,37
mer_numerical_1,23.47,1.093,0.02185,-0.05746,23.47
mer_numerical_2,23.02,-0.05746,0.02185,-0.05746,23.02
mer_category_1,1,1,1,1,1
mer_most_recent_sales_range,0,0,2,3,0
mer_most_recent_purchases_range,0,0,2,2,0


In [50]:
merTrans.to_csv('merTrans.csv')

In [5]:
merTrans = reduce_mem_usage(pd.read_csv("merTrans.csv"))

Memory usage after optimization is: 2158.17 MB
Decreased by 63.5%


In [6]:
merTrans.head().T

,0,1,2,3,4
Unnamed: 0,0,1,2,3,4
card_id,C_ID_4e6213e9bc,C_ID_4e6213e9bc,C_ID_4e6213e9bc,C_ID_4e6213e9bc,C_ID_4e6213e9bc
merchant_id,M_ID_e020e9b302,M_ID_86ec983688,M_ID_979ed661fc,M_ID_e6d5ae8ea6,M_ID_e020e9b302
mer_merchant_group_id,35,2084,27369,24104,35
mer_merchant_category_id,80,367,80,560,80
mer_subsector_id,37,16,37,34,37
mer_numerical_1,23.4688,1.09277,0.0218506,-0.0574646,23.4688
mer_numerical_2,23.0156,-0.0574646,0.0218506,-0.0574646,23.0156
mer_category_1,1,1,1,1,1
mer_most_recent_sales_range,0,0,2,3,0


In [8]:
common_aggs = ['sum', 'mean', 'std']
common_aggs1 = ['sum', 'mean', 'min', 'max', 'std']
aggs = {
      #Numerical features 
      'mer_numerical_1' : common_aggs1,
      'mer_numerical_2' : common_aggs1,
      'mer_avg_sales_lag3' : common_aggs1,
      'mer_avg_sales_lag6' : common_aggs, 
      'mer_avg_sales_lag12' : common_aggs1, 
      'mer_avg_purchases_lag3' : common_aggs1,
      'mer_avg_purchases_lag6' : common_aggs1,
      'mer_avg_purchases_lag12' : common_aggs1,
      'mer_active_months_lag3' : common_aggs1,
      'mer_active_months_lag6' : common_aggs1,
      'mer_active_months_lag12' : common_aggs1, 

      #categorical features 
      'mer_category_1' : common_aggs, 
      'mer_most_recent_sales_range' : common_aggs,
      'mer_most_recent_purchases_range' : common_aggs,
      'mer_category_4' : common_aggs,
      'mer_category_2' : common_aggs
}

In [9]:
%%time
merfeat = merTrans.groupby('card_id').agg(aggs)
#rename 
merfeat.columns = ['_'.join(col).strip() 
                           for col in merfeat.columns.values]

CPU times: user 32.1 s, sys: 331 ms, total: 32.4 s
Wall time: 32.5 s


In [10]:
# store the features in a csv file 
merfeat.to_csv('merfeat.csv')

In [11]:
merfeat

,mer_numerical_1_sum,mer_numerical_1_mean,mer_numerical_1_min,mer_numerical_1_max,mer_numerical_1_std,mer_numerical_2_sum,mer_numerical_2_mean,mer_numerical_2_min,mer_numerical_2_max,mer_numerical_2_std,mer_avg_sales_lag3_sum,mer_avg_sales_lag3_mean,mer_avg_sales_lag3_min,mer_avg_sales_lag3_max,mer_avg_sales_lag3_std,mer_avg_sales_lag6_sum,mer_avg_sales_lag6_mean,mer_avg_sales_lag6_std,mer_avg_sales_lag12_sum,mer_avg_sales_lag12_mean,mer_avg_sales_lag12_min,mer_avg_sales_lag12_max,mer_avg_sales_lag12_std,mer_avg_purchases_lag3_sum,mer_avg_purchases_lag3_mean,mer_avg_purchases_lag3_min,mer_avg_purchases_lag3_max,mer_avg_purchases_lag3_std,mer_avg_purchases_lag6_sum,mer_avg_purchases_lag6_mean,mer_avg_purchases_lag6_min,mer_avg_purchases_lag6_max,mer_avg_purchases_lag6_std,mer_avg_purchases_lag12_sum,mer_avg_purchases_lag12_mean,mer_avg_purchases_lag12_min,mer_avg_purchases_lag12_max,mer_avg_purchases_lag12_std,mer_active_months_lag3_sum,mer_active_months_lag3_mean,mer_active_months_lag3_min,mer_active_months_lag3_max,mer_active_months_lag3_std,mer_active_months_lag6_sum,mer_active_months_lag6_mean,mer_active_months_lag6_min,mer_active_months_lag6_max,mer_active_months_lag6_std,mer_active_months_lag12_sum,mer_active_months_lag12_mean,mer_active_months_lag12_min,mer_active_months_lag12_max,mer_active_months_lag12_std,mer_category_1_sum,mer_category_1_mean,mer_category_1_std,mer_most_recent_sales_range_sum,mer_most_recent_sales_range_mean,mer_most_recent_sales_range_std,mer_most_recent_purchases_range_sum,mer_most_recent_purchases_range_mean,mer_most_recent_purchases_range_std,mer_category_4_sum,mer_category_4_mean,mer_category_4_std,mer_category_2_sum,mer_category_2_mean,mer_category_2_std
card_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
C_ID_00007093c1,270.000000,1.776367,-0.057465,15.859375,4.376036,267.000000,1.756836,-0.057465,15.828125,4.369701,152.680008,1.004474,0.62,6.93,0.498170,161.149994,1.060197,0.616332,169.599991,1.115789,0.54,8.570000,0.689795,163.0000,1.072266,0.762695,12.703125,0.953946,174.25000,1.146484,0.717773,15.859375,1.219216,184.000000,1.210938,0.549805,16.21875,1.333640,456.0,3.0,3.0,3.0,0.0,912.0,6.0,6.0,6.0,0.0,1807.0,11.890625,7.0,12.0,0.625605,122.0,0.802734,0.399328,275.0,1.809570,1.472636,289.0,1.901367,1.445522,100.0,0.657715,0.475983,396.0,2.605469,0.798657
C_ID_0001238066,1324.000000,8.882812,-0.057465,129.375000,30.211845,1312.000000,8.804688,-0.057465,128.500000,29.971588,177.619995,1.192080,0.77,16.18,1.353614,336.809998,2.260470,12.173580,368.600006,2.473825,0.53,166.679993,13.840760,187.5000,1.257812,0.811035,12.703125,1.453369,697.50000,4.679688,0.738281,504.250000,41.244159,757.000000,5.082031,0.540527,554.50000,45.371265,447.0,3.0,3.0,3.0,0.0,894.0,6.0,6.0,6.0,0.0,1780.0,11.945312,7.0,12.0,0.446990,138.0,0.926270,0.262369,269.0,1.805664,1.044146,249.0,1.670898,1.105532,119.0,0.798828,0.402356,233.0,1.563477,1.396531
C_ID_0001506ef0,65.500000,0.949707,-0.057465,15.390625,2.570203,56.468750,0.818359,-0.057465,15.109375,2.564777,83.139999,1.204928,0.72,6.93,1.001999,87.150002,1.263044,1.251992,88.459999,1.282029,0.76,8.570000,1.278745,96.1875,1.393555,0.822266,12.703125,1.969871,104.31250,1.511719,0.871094,15.859375,2.498941,106.625000,1.545898,0.752441,16.21875,2.556361,207.0,3.0,3.0,3.0,0.0,414.0,6.0,6.0,6.0,0.0,818.0,11.851562,7.0,12.0,0.844974,65.0,0.941895,0.235401,92.0,1.333008,1.268008,91.0,1.318359,1.300370,1.0,0.014496,0.120386,199.0,2.884766,0.470801
C_ID_0001793786,287.250000,1.134766,-0.057465,117.812500,8.175810,275.500000,1.088867,-0.057465,116.875000,8.113782,343.700012,1.358498,0.55,26.91,1.971153,582.750000,2.303360,10.823875,610.869995,2.414506,0.50,166.679993,11.507896,437.0000,1.726562,0.549316,87.687500,5.743095,1146.00000,4.531250,0.630859,504.250000,35.683157,1193.000000,4.714844,0.437988,554.50000,37.848444,759.0,3.0,3.0,3.0,0.0,1518.0,6.0,6.0,6.0,0.0,2996.0,11.843750,7.0,12.0,0.804225,217.0,0.857910,0.350042,503.0,1.988281,1.190179,520.0,2

## Merge everything 

In [12]:
#read all the features 
dateTimeFeatures = reduce_mem_usage(pd.read_csv("dateTimeFeatures.csv"))
otherFeatures = reduce_mem_usage(pd.read_csv("otherFeatures.csv"))

Memory usage after optimization is: 79.79 MB
Decreased by 72.3%
Memory usage after optimization is: 20.80 MB
Decreased by 70.1%


In [22]:
#load train and test 
train = reduce_mem_usage(pd.read_csv("train.csv"))
test = reduce_mem_usage(pd.read_csv("test.csv"))

Memory usage after optimization is: 4.04 MB
Decreased by 56.2%
Memory usage after optimization is: 2.24 MB
Decreased by 52.5%


In [23]:
train

,first_active_month,card_id,feature_1,feature_2,feature_3,target
0,2017-06,C_ID_92a2005557,5,2,1,-0.820312
1,2017-01,C_ID_3d0044924f,4,1,0,0.392822
2,2016-08,C_ID_d639edf6cd,2,2,0,0.687988
3,2017-09,C_ID_186d6a6901,4,3,0,0.142456
4,2017-11,C_ID_cdbd2c0db2,1,3,0,-0.159790
...,...,...,...,...,...,...
201912,2017-09,C_ID_963962de2c,3,2,1,-2.740234
201913,2015-10,C_ID_1314773c0b,3,1,1,0.312988
201914,2017-08,C_ID_7666735b3d,4,3,0,0.093506
201915,2016-07,C_ID_73f5a0efd0,3,2,1,-4.675781


In [24]:
test

,first_active_month,card_id,feature_1,feature_2,feature_3
0,2017-04,C_ID_0ab67a22ab,3,3,1
1,2017-01,C_ID_130fd0cbdd,2,3,0
2,2017-08,C_ID_b709037bc5,5,1,1
3,2017-12,C_ID_d27d835a9f,2,1,0
4,2015-12,C_ID_2b5e3df5c2,5,1,1
...,...,...,...,...,...
123618,2017-10,C_ID_7a239d2eda,2,2,0
123619,2017-09,C_ID_75ace375ae,3,1,1
123620,2016-09,C_ID_21d56d950c,5,1,1
123621,2017-06,C_ID_6c46fc5a9d,2,1,0


In [25]:

def featureEngineering(df):
  #some feature engineering based on the main train set features 
  df['first_active_month'] = pd.to_datetime(df['first_active_month'])

  # datetime features
  df['year'] =  df['first_active_month'].dt.year
  df['quarter'] = df['first_active_month'].dt.quarter
  df['elapsed_time'] = (datetime.datetime.today() - df['first_active_month']).dt.days

  # Some basic statistics transformations over the feature_i columns
  df['feature_sum'] = df['feature_1'] + df['feature_2'] + df['feature_3']
  df['feature_mean'] = df['feature_sum'] / 3
  df['feature_max'] = df[['feature_1', 'feature_2', 'feature_3']].max(axis=1)
  df['feature_min'] = df[['feature_1', 'feature_2', 'feature_3']].min(axis=1)
  df['feature_std'] = df[['feature_1', 'feature_2', 'feature_3']].std(axis=1)

  return df 


In [26]:
train = featureEngineering(train)

In [27]:
train

,first_active_month,card_id,feature_1,feature_2,feature_3,target,year,quarter,elapsed_time,feature_sum,feature_mean,feature_max,feature_min,feature_std
0,2017-06-01,C_ID_92a2005557,5,2,1,-0.820312,2017,2,1357,8,2.666667,5,1,2.081666
1,2017-01-01,C_ID_3d0044924f,4,1,0,0.392822,2017,1,1508,5,1.666667,4,0,2.081666
2,2016-08-01,C_ID_d639edf6cd,2,2,0,0.687988,2016,3,1661,4,1.333333,2,0,1.154701
3,2017-09-01,C_ID_186d6a6901,4,3,0,0.142456,2017,3,1265,7,2.333333,4,0,2.081666
4,2017-11-01,C_ID_cdbd2c0db2,1,3,0,-0.159790,2017,4,1204,4,1.333333,3,0,1.527525
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201912,2017-09-01,C_ID_963962de2c,3,2,1,-2.740234,2017,3,1265,6,2.000000,3,1,1.000000
201913,2015-10-01,C_ID_1314773c0b,3,1,1,0.312988,2015,4,1966,5,1.666667,3,1,1.154701
201914,2017-08-01,C_ID_7666735b3d,4,3,0,0.093506,2017,3,1296,7,2.333333,4,0,2.081666
201915,2016-07-01,C_ID_73f5a0efd0,3,2,1,-4.675781,2016,3,1692,6,2.000000,3,1,1.000000


In [28]:
test = featureEngineering(test)

In [29]:
test

,first_active_month,card_id,feature_1,feature_2,feature_3,year,quarter,elapsed_time,feature_sum,feature_mean,feature_max,feature_min,feature_std
0,2017-04-01,C_ID_0ab67a22ab,3,3,1,2017.0,2.0,1418.0,7,2.333333,3,1,1.154701
1,2017-01-01,C_ID_130fd0cbdd,2,3,0,2017.0,1.0,1508.0,5,1.666667,3,0,1.527525
2,2017-08-01,C_ID_b709037bc5,5,1,1,2017.0,3.0,1296.0,7,2.333333,5,1,2.309401
3,2017-12-01,C_ID_d27d835a9f,2,1,0,2017.0,4.0,1174.0,3,1.000000,2,0,1.000000
4,2015-12-01,C_ID_2b5e3df5c2,5,1,1,2015.0,4.0,1905.0,7,2.333333,5,1,2.309401
...,...,...,...,...,...,...,...,...,...,...,...,...,...
123618,2017-10-01,C_ID_7a239d2eda,2,2,0,2017.0,4.0,1235.0,4,1.333333,2,0,1.154701
123619,2017-09-01,C_ID_75ace375ae,3,1,1,2017.0,3.0,1265.0,5,1.666667,3,1,1.154701
123620,2016-09-01,C_ID_21d56d950c,5,1,1,2016.0,3.0,1630.0,7,2.333333,5,1,2.309401
123621,2017-06-01,C_ID_6c46fc5a9d,2,1,0,2017.0,2.0,1357.0,3,1.000000,2,0,1.000000


In [30]:
#now merge the features on the basis of card id 

#date time features 
train = pd.merge(train, dateTimeFeatures,on='card_id', how='left' )
test = pd.merge(test, dateTimeFeatures,on='card_id', how='left' )

In [31]:
train

,first_active_month,card_id,feature_1,feature_2,feature_3,target,year,quarter,elapsed_time,feature_sum,feature_mean,feature_max,feature_min,feature_std,purchase_Year_mode,purchase_Week_sum,purchase_Week_mean,purchase_Week_std,purchase_Day_sum,purchase_Day_mean,purchase_Day_std,purchase_Dayofyear_sum,purchase_Dayofyear_mean,purchase_Dayofyear_std,purchase_Is_month_end_sum,purchase_Is_month_end_mean,purchase_Is_month_end_std,purchase_Is_month_start_sum,purchase_Is_month_start_mean,purchase_Is_month_start_std,purchase_Is_quarter_end_sum,purchase_Is_quarter_end_mean,purchase_Is_quarter_end_std,purchase_Is_quarter_start_sum,purchase_Is_quarter_start_mean,purchase_Is_quarter_start_std,purchase_Is_year_end_sum,purchase_Is_year_end_mean,purchase_Is_year_end_std,purchase_Is_year_start_sum,...,purchase_Month_10_std,purchase_Month_11_sum,purchase_Month_11_mean,purchase_Month_11_std,purchase_Month_12_sum,purchase_Month_12_mean,purchase_Month_12_std,purchasedWeekend_0_sum,purchasedWeekend_0_mean,purchasedWeekend_0_std,purchasedWeekend_1_sum,purchasedWeekend_1_mean,purchasedWeekend_1_std,purchase_Dayofweek_0_sum,purchase_Dayofweek_0_mean,purchase_Dayofweek_0_std,purchase_Dayofweek_1_sum,purchase_Dayofweek_1_mean,purchase_Dayofweek_1_std,purchase_Dayofweek_2_sum,purchase_Dayofweek_2_mean,purchase_Dayofweek_2_std,purchase_Dayofweek_3_sum,purchase_Dayofweek_3_mean,purchase_Dayofweek_3_std,purchase_Dayofweek_4_sum,purchase_Dayofweek_4_mean,purchase_Dayofweek_4_std,purchase_Dayofweek_5_sum,purchase_Dayofweek_5_mean,purchase_Dayofweek_5_std,purchase_Dayofweek_6_sum,purchase_Dayofweek_6_mean,purchase_Dayofweek_6_std,purchase_date_temp_period,purchase_date_temp_mean_diff,purchase_date_temp_min,purchase_date_temp_max,monthDiff_period,monthDiff_mean_diff
0,2017-06-01,C_ID_92a2005557,5,2,1,-0.820312,2017,2,1357,8,2.666667,5,1,2.081666,2017,8905,31.468750,15.554688,4411,15.585938,8.804688,61760,218.250,109.1875,9,0.031799,0.175781,7,0.024734,0.155640,5,0.017670,0.131958,3,0.010597,0.102600,1,0.003534,0.059448,0,...,0.268311,21.0,0.074219,0.262451,57.0,0.201416,0.401855,187.0,0.660645,0.474365,96.0,0.339111,0.474365,34.0,0.120117,0.325684,33.0,0.116638,0.321533,44.0,0.155518,0.363037,41.0,0.144897,0.352539,35.0,0.123657,0.329834,53.0,0.187256,0.390869,43.0,0.151978,0.359619,306,26.046875,1331,1025,1,0.003546
1,2017-01-01,C_ID_3d0044924f,4,1,0,0.392822,2017,1,1508,5,1.666667,4,0,2.081666,2017,8881,24.953125,16.703125,5914,16.609375,8.835938,61516,172.750,117.1875,12,0.033722,0.180786,15,0.042145,0.201172,6,0.016861,0.128906,6,0.016861,0.128906,3,0.008430,0.091553,1,...,0.260498,16.0,0.044952,0.207520,47.0,0.132080,0.339111,224.0,0.629395,0.483643,132.0,0.370850,0.483643,42.0,0.117981,0.322998,47.0,0.132080,0.339111,35.0,0.098328,0.298096,44.0,0.123596,0.329590,56.0,0.157349,0.364502,79.0,0.221924,0.416016,53.0,0.148926,0.356445,448,30.281250,1503,1055,1,0.002817
2,2016-08-01,C_ID_d639edf6cd,2,2,0,0.687988,2016,3,1661,4,1.333333,2,0,1.154701,2017,807,18.343750,13.726562,859,19.515625,7.910156,5572,126.625,96.5000,1,0.022720,0.150757,0,0.000000,0.000000,0,0.000000,0.000000,0,0.000000,0.000000,0,0.000000,0.000000,0,...,0.347168,0.0,0.000000,0.000000,1.0,0.022720,0.150757,32.0,0.727051,0.450439,12.0,0.272705,0.450439,3.0,0.068176,0.254883,2.0,0.045441,0.210693,9.0,0.204590,0.407959,7.0,0.159058,0.369873,11.0,0.250000,0.437988,10.0,0.227295,0.423828,2.0,0.045441,0.210693,472,263.500000,1498,1026,1,0.023254
3,2017-09-01,C_ID_186d6a6901,4,3,0,0.142456,2017,3,1265,7,2.333333,4,0,2.081666,2017,2562,30.500000,17.140625,1391,16.562500,8.906250,17723,211.000,120.3750,3,0.035706,0.186646,0,0.000000,0.000000,0,0.000000,0.000000,0,0.000000,0.000000,0,0.000000,0.000000,0,...,0.485352,7.0,0.083313,0.278076,6.0,0.071411,0.259033,70.0,0.833496,0.375000,14.0,0.166626,0.375000,7.0,0.083313,0.278076,12.0,0.142822,0.352051,22.0,0.261963,0.442383,10.0,0.119019,0.325684,19.0,0.226196,0.420898,8.0,0.095215,0.295410,6.0,0.071411,0.259033,204,59.000000,1240,1036,1,0.012047
4,2017-1

In [32]:
#otherFeatures
train = pd.merge(train, otherFeatures,on='card_id', how='left' )
test = pd.merge(test, otherFeatures,on='card_id', how='left' )

In [33]:
train

,first_active_month,card_id,feature_1,feature_2,feature_3,target,year,quarter,elapsed_time,feature_sum,feature_mean,feature_max,feature_min,feature_std,purchase_Year_mode,purchase_Week_sum,purchase_Week_mean,purchase_Week_std,purchase_Day_sum,purchase_Day_mean,purchase_Day_std,purchase_Dayofyear_sum,purchase_Dayofyear_mean,purchase_Dayofyear_std,purchase_Is_month_end_sum,purchase_Is_month_end_mean,purchase_Is_month_end_std,purchase_Is_month_start_sum,purchase_Is_month_start_mean,purchase_Is_month_start_std,purchase_Is_quarter_end_sum,purchase_Is_quarter_end_mean,purchase_Is_quarter_end_std,purchase_Is_quarter_start_sum,purchase_Is_quarter_start_mean,purchase_Is_quarter_start_std,purchase_Is_year_end_sum,purchase_Is_year_end_mean,purchase_Is_year_end_std,purchase_Is_year_start_sum,...,purchase_Dayofweek_4_std,purchase_Dayofweek_5_sum,purchase_Dayofweek_5_mean,purchase_Dayofweek_5_std,purchase_Dayofweek_6_sum,purchase_Dayofweek_6_mean,purchase_Dayofweek_6_std,purchase_date_temp_period,purchase_date_temp_mean_diff,purchase_date_temp_min,purchase_date_temp_max,monthDiff_period,monthDiff_mean_diff,authorized_flag_sum,authorized_flag_mean,authorized_flag_std,category_1_sum,category_1_mean,category_1_std,category_2_sum,category_2_mean,category_2_std,category_3_sum,category_3_mean,category_3_std,month_lag_sum,month_lag_mean,month_lag_min,month_lag_max,month_lag_std,installments_sum,installments_mean,installments_min,installments_max,installments_std,purchase_amount_sum,purchase_amount_mean,purchase_amount_min,purchase_amount_max,purchase_amount_std
0,2017-06-01,C_ID_92a2005557,5,2,1,-0.820312,2017,2,1357,8,2.666667,5,1,2.081666,2017,8905,31.468750,15.554688,4411,15.585938,8.804688,61760,218.250,109.1875,9,0.031799,0.175781,7,0.024734,0.155640,5,0.017670,0.131958,3,0.010597,0.102600,1,0.003534,0.059448,0,...,0.329834,53.0,0.187256,0.390869,43.0,0.151978,0.359619,306,26.046875,1331,1025,1,0.003546,270.0,0.954102,0.209717,0.0,0.000000,0.000000,295.0,1.041992,0.410400,4.0,0.014137,0.118225,-983.0,-3.472656,-8,2,2.734375,4,0.014137,0,1,0.118225,21403.699219,75.631447,5.000000,2000.000000,138.101593
1,2017-01-01,C_ID_3d0044924f,4,1,0,0.392822,2017,1,1508,5,1.666667,4,0,2.081666,2017,8881,24.953125,16.703125,5914,16.609375,8.835938,61516,172.750,117.1875,12,0.033722,0.180786,15,0.042145,0.201172,6,0.016861,0.128906,6,0.016861,0.128906,3,0.008430,0.091553,1,...,0.364502,79.0,0.221924,0.416016,53.0,0.148926,0.356445,448,30.281250,1503,1055,1,0.002817,345.0,0.969238,0.173340,31.0,0.087097,0.282227,356.0,1.000000,0.000000,426.0,1.196289,0.411865,-1752.0,-4.921875,-12,2,3.865234,549,1.541992,-1,10,1.500000,34297.628906,96.341660,3.000000,3578.479980,254.249817
2,2016-08-01,C_ID_d639edf6cd,2,2,0,0.687988,2016,3,1661,4,1.333333,2,0,1.154701,2017,807,18.343750,13.726562,859,19.515625,7.910156,5572,126.625,96.5000,1,0.022720,0.150757,0,0.000000,0.000000,0,0.000000,0.000000,0,0.000000,0.000000,0,0.000000,0.000000,0,...,0.437988,10.0,0.227295,0.423828,2.0,0.045441,0.210693,472,263.500000,1498,1026,1,0.023254,42.0,0.954590,0.210693,0.0,0.000000,0.000000,204.0,4.636719,1.163086,0.0,0.000000,0.000000,-368.0,-8.367188,-13,2,4.121094,0,0.000000,0,0,0.000000,1994.050049,45.319317,11.156250,400.000000,57.512569
3,2017-09-01,C_ID_186d6a6901,4,3,0,0.142456,2017,3,1265,7,2.333333,4,0,2.081666,2017,2562,30.500000,17.140625,1391,16.562500,8.906250,17723,211.000,120.3750,3,0.035706,0.186646,0,0.000000,0.000000,0,0.000000,0.000000,0,0.000000,0.000000,0,0.000000,0.000000,0,...,0.420898,8.0,0.095215,0.295410,6.0,0.071411,0.259033,204,59.000000,1240,1036,1,0.012047,84.0,1.000000,0.000000,13.0,0.154785,0.363770,261.0,3.107422,1.379883,88.0,1.047852,0.343750,-206.0,-2.453125,-5,2,2.142578,89,1.059570,-1,3,0.608398,5719.299805,68.086906,4.000000,1459.089966,167.072281
4,2017-11-01,C_ID_cdbd2c0db2,1,3,0,-0.159790,2017,4,1204,4,1.333333,3,0,1.527525,2018,3378,19.984375,19.593750,2666,15.773438,8.953125,23233,137.500,137.6250,8,0.047333,0.213013,7,0.041412,0.199829,2,0.01183

In [34]:
#merchant features 
train = pd.merge(train, merfeat,on='card_id', how='left' )
test = pd.merge(test, merfeat,on='card_id', how='left' )

In [35]:
train

,first_active_month,card_id,feature_1,feature_2,feature_3,target,year,quarter,elapsed_time,feature_sum,feature_mean,feature_max,feature_min,feature_std,purchase_Year_mode,purchase_Week_sum,purchase_Week_mean,purchase_Week_std,purchase_Day_sum,purchase_Day_mean,purchase_Day_std,purchase_Dayofyear_sum,purchase_Dayofyear_mean,purchase_Dayofyear_std,purchase_Is_month_end_sum,purchase_Is_month_end_mean,purchase_Is_month_end_std,purchase_Is_month_start_sum,purchase_Is_month_start_mean,purchase_Is_month_start_std,purchase_Is_quarter_end_sum,purchase_Is_quarter_end_mean,purchase_Is_quarter_end_std,purchase_Is_quarter_start_sum,purchase_Is_quarter_start_mean,purchase_Is_quarter_start_std,purchase_Is_year_end_sum,purchase_Is_year_end_mean,purchase_Is_year_end_std,purchase_Is_year_start_sum,...,mer_avg_purchases_lag6_sum,mer_avg_purchases_lag6_mean,mer_avg_purchases_lag6_min,mer_avg_purchases_lag6_max,mer_avg_purchases_lag6_std,mer_avg_purchases_lag12_sum,mer_avg_purchases_lag12_mean,mer_avg_purchases_lag12_min,mer_avg_purchases_lag12_max,mer_avg_purchases_lag12_std,mer_active_months_lag3_sum,mer_active_months_lag3_mean,mer_active_months_lag3_min,mer_active_months_lag3_max,mer_active_months_lag3_std,mer_active_months_lag6_sum,mer_active_months_lag6_mean,mer_active_months_lag6_min,mer_active_months_lag6_max,mer_active_months_lag6_std,mer_active_months_lag12_sum,mer_active_months_lag12_mean,mer_active_months_lag12_min,mer_active_months_lag12_max,mer_active_months_lag12_std,mer_category_1_sum,mer_category_1_mean,mer_category_1_std,mer_most_recent_sales_range_sum,mer_most_recent_sales_range_mean,mer_most_recent_sales_range_std,mer_most_recent_purchases_range_sum,mer_most_recent_purchases_range_mean,mer_most_recent_purchases_range_std,mer_category_4_sum,mer_category_4_mean,mer_category_4_std,mer_category_2_sum,mer_category_2_mean,mer_category_2_std
0,2017-06-01,C_ID_92a2005557,5,2,1,-0.820312,2017,2,1357,8,2.666667,5,1,2.081666,2017,8905,31.468750,15.554688,4411,15.585938,8.804688,61760,218.250,109.1875,9,0.031799,0.175781,7,0.024734,0.155640,5,0.017670,0.131958,3,0.010597,0.102600,1,0.003534,0.059448,0,...,5484.00000,19.437500,0.321045,504.250000,93.301270,6044.00000,21.437500,0.252930,554.500000,102.781945,846.0,3.000000,3.0,3.0,0.000000,1691.0,5.996094,5.0,6.0,0.059549,3348.0,11.875000,5.0,12.0,0.794564,258.0,0.915039,0.279536,469.0,1.663086,1.278061,410.0,1.454102,1.287637,16.0,0.056732,0.231752,294.0,1.042969,0.411098
1,2017-01-01,C_ID_3d0044924f,4,1,0,0.392822,2017,1,1508,5,1.666667,4,0,2.081666,2017,8881,24.953125,16.703125,5914,16.609375,8.835938,61516,172.750,117.1875,12,0.033722,0.180786,15,0.042145,0.201172,6,0.016861,0.128906,6,0.016861,0.128906,3,0.008430,0.091553,1,...,1546.00000,4.167969,0.307861,504.250000,36.939265,1679.00000,4.527344,0.208984,554.500000,40.632763,1113.0,3.000000,3.0,3.0,0.000000,2226.0,6.000000,6.0,6.0,0.000000,4400.0,11.859375,7.0,12.0,0.788411,309.0,0.833008,0.373583,403.0,1.085938,1.362867,397.0,1.070312,1.363796,29.0,0.078186,0.268797,371.0,1.000000,0.000000
2,2016-08-01,C_ID_d639edf6cd,2,2,0,0.687988,2016,3,1661,4,1.333333,2,0,1.154701,2017,807,18.343750,13.726562,859,19.515625,7.910156,5572,126.625,96.5000,1,0.022720,0.150757,0,0.000000,0.000000,0,0.000000,0.000000,0,0.000000,0.000000,0,0.000000,0.000000,0,...,61.96875,1.376953,0.871094,15.859375,2.209275,62.84375,1.396484,0.752441,16.218750,2.261855,135.0,3.000000,3.0,3.0,0.000000,270.0,6.000000,6.0,6.0,0.000000,533.0,11.843750,7.0,12.0,0.796457,43.0,0.955566,0.208409,18.0,0.399902,0.962950,18.0,0.399902,1.009050,3.0,0.066650,0.252262,205.0,4.554688,1.271283
3,2017-09-01,C_ID_186d6a6901,4,3,0,0.142456,2017,3,1265,7,2.333333,4,0,2.081666,2017,2562,30.500000,17.140625,1391,16.562500,8.906250,17723,211.000,120.3750,3,0.035706,0.186646,0,0.000000,0.000000,0,0.000000,0.000000,0,0.000000,0.000000,0,0.000000,0.000000,0,...,230.87500,2.404297,0.426270,15.859375,4.353584,234.12500,2.437500,0.436035,16.218750,4.461519,288.0,3.000000,3.0,3.0,0.000000,576.0,6.000

In [36]:
test

,first_active_month,card_id,feature_1,feature_2,feature_3,year,quarter,elapsed_time,feature_sum,feature_mean,feature_max,feature_min,feature_std,purchase_Year_mode,purchase_Week_sum,purchase_Week_mean,purchase_Week_std,purchase_Day_sum,purchase_Day_mean,purchase_Day_std,purchase_Dayofyear_sum,purchase_Dayofyear_mean,purchase_Dayofyear_std,purchase_Is_month_end_sum,purchase_Is_month_end_mean,purchase_Is_month_end_std,purchase_Is_month_start_sum,purchase_Is_month_start_mean,purchase_Is_month_start_std,purchase_Is_quarter_end_sum,purchase_Is_quarter_end_mean,purchase_Is_quarter_end_std,purchase_Is_quarter_start_sum,purchase_Is_quarter_start_mean,purchase_Is_quarter_start_std,purchase_Is_year_end_sum,purchase_Is_year_end_mean,purchase_Is_year_end_std,purchase_Is_year_start_sum,purchase_Is_year_start_mean,...,mer_avg_purchases_lag6_sum,mer_avg_purchases_lag6_mean,mer_avg_purchases_lag6_min,mer_avg_purchases_lag6_max,mer_avg_purchases_lag6_std,mer_avg_purchases_lag12_sum,mer_avg_purchases_lag12_mean,mer_avg_purchases_lag12_min,mer_avg_purchases_lag12_max,mer_avg_purchases_lag12_std,mer_active_months_lag3_sum,mer_active_months_lag3_mean,mer_active_months_lag3_min,mer_active_months_lag3_max,mer_active_months_lag3_std,mer_active_months_lag6_sum,mer_active_months_lag6_mean,mer_active_months_lag6_min,mer_active_months_lag6_max,mer_active_months_lag6_std,mer_active_months_lag12_sum,mer_active_months_lag12_mean,mer_active_months_lag12_min,mer_active_months_lag12_max,mer_active_months_lag12_std,mer_category_1_sum,mer_category_1_mean,mer_category_1_std,mer_most_recent_sales_range_sum,mer_most_recent_sales_range_mean,mer_most_recent_sales_range_std,mer_most_recent_purchases_range_sum,mer_most_recent_purchases_range_mean,mer_most_recent_purchases_range_std,mer_category_4_sum,mer_category_4_mean,mer_category_4_std,mer_category_2_sum,mer_category_2_mean,mer_category_2_std
0,2017-04-01,C_ID_0ab67a22ab,3,3,1,2017.0,2.0,1418.0,7,2.333333,3,1,1.154701,2017,2370,33.375000,11.820312,1159,16.328125,9.210938,16433,231.5000,82.93750,5,0.070435,0.257568,3,0.042267,0.202637,2,0.028168,0.166626,0,0.000000,0.000000,0,0.0,0.0,0,0.0,...,123.750000,1.671875,0.783203,15.859375,3.018042,119.375000,1.613281,0.583984,16.218750,3.099904,222.0,3.0,3.0,3.0,0.0,444.0,6.000000,6.0,6.0,0.000000,873.0,11.796875,7.0,12.0,0.992847,45.0,0.607910,0.491505,124.0,1.675781,1.405548,125.0,1.689453,1.461147,39.0,0.526855,0.502677,74.0,1.000000,0.000000
1,2017-01-01,C_ID_130fd0cbdd,2,3,0,2017.0,1.0,1508.0,5,1.666667,3,0,1.527525,2017,1082,12.437500,5.433594,1390,15.976562,7.300781,7384,84.8750,37.96875,0,0.000000,0.000000,0,0.000000,0.000000,0,0.000000,0.000000,0,0.000000,0.000000,0,0.0,0.0,0,0.0,...,106.375000,1.208984,0.859863,15.859375,1.582818,109.250000,1.241211,0.710449,16.218750,1.655591,264.0,3.0,3.0,3.0,0.0,528.0,6.000000,6.0,6.0,0.000000,1047.0,11.898438,7.0,12.0,0.678737,83.0,0.943359,0.232822,155.0,1.761719,1.093285,151.0,1.715820,1.092807,82.0,0.931641,0.253502,337.0,3.830078,0.698465
2,2017-08-01,C_ID_b709037bc5,5,1,1,2017.0,3.0,1296.0,7,2.333333,5,1,2.309401,2017,397,26.468750,20.687500,125,8.335938,7.343750,2735,182.3750,144.62500,0,0.000000,0.000000,1,0.066650,0.258301,0,0.000000,0.000000,0,0.000000,0.000000,0,0.0,0.0,0,0.0,...,16.515625,1.100586,0.632324,1.474609,0.228659,16.578125,1.104492,0.583984,1.452148,0.266767,45.0,3.0,3.0,3.0,0.0,90.0,6.000000,6.0,6.0,0.000000,180.0,12.000000,12.0,12.0,0.000000,13.0,0.866699,0.351866,26.0,1.733398,0.798809,37.0,2.466797,1.060099,3.0,0.199951,0.414039,67.0,4.464844,1.407463
3,2017-12-01,C_ID_d27d835a9f,2,1,0,2017.0,4.0,1174.0,3,1.000000,2,0,1.000000,2018,930,25.828125,20.203125,433,12.031250,7.343750,6405,177.8750,141.87500,1,0.027771,0.166626,0,0.000000,0.000000,1,0.027771,0.166626,0,0.000000,0.000000,0,0.0,0.0,0,0.0,...,36.843750,1.023438,0.783203,1.226562,0.092627,38.500000,1.069336,0.583984,1.327148,0.136481,108.0,3.0,3.0,3.0,0.0,216.0,6.000000,6.0,6.0,0.000000,432.0,12.000000,12.0,12.0,0.000000,33.0,0.916504,0.280306,4

In [37]:
train.to_csv('FINAL_TRAIN_M.csv')
test.to_csv('FINAL_TEST_M.csv')